In [1]:
from deepst.datasets.TaxiBJ import load_data
from deepst.config import Config
from deepst.models.STResNet import stresnet
from deepst.metrics import rmse
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
C:\Users\gunarton\AppData\Local\Continuum\anaconda3\envs\py3\lib\site-packages\google\protobuf\descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from google.protobuf.pyext import _message
C:\Users\gunarton\AppData\Local\Continuum\anaconda3\envs\py3\lib\site-packages\tensorflow\python\util\nest.py:1287: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  _pywrap_tensorflow.RegisterType("Sequence", _collections.Sequence)
C:\Users\gunarton\AppData\Local\Continuum\anaconda3\envs\py3\lib\site-packages\tensorflow\python\training\tracking\object_identity.py:61: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class ObjectIdentityDictionary(collections.Muta

In [2]:
config = Config()

In [3]:
### Dataset configuration
N_FLOW = 2
GRID_HEIGHT = 32
GRID_WIDTH = 32

In [4]:
### Hyperparameters (Can be tuned)
N_TEST = 48 * 5 * 4 # 4 weeks -- somehow
N_CLOSE = 3    # {3, 4, 5}
N_PERIOD = 3   # {1, 2, 3, 4}
N_TREND = 3    # {1, 2, 3, 4}

N_RESIDUAL = 12

In [5]:
c_conf = (N_CLOSE, N_FLOW, GRID_HEIGHT, GRID_WIDTH)
p_conf = (N_PERIOD, N_FLOW, GRID_HEIGHT, GRID_WIDTH)
t_conf = (N_TREND, N_FLOW, GRID_HEIGHT, GRID_WIDTH)

In [ ]:
X_train, Y_train, X_test, Y_test, mmn, metadata_dim, timestamp_train, timestamp_test = load_data(len_closeness=N_CLOSE, len_period=N_PERIOD, len_trend=N_TREND, len_test=N_TEST)

In [ ]:
### Evaluate the testing time window (4 weeks)
timestamp_test[0], timestamp_test[-1]

In [ ]:
model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf, external_dim=metadata_dim, nb_residual_unit=N_RESIDUAL)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmse])
model.summary()

In [ ]:
### Execute the training process
batch_size = 32
epochs = 5
history = model.fit(X_train, Y_train, validation_split=0.1, epochs=epochs, batch_size=batch_size)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
_, result = model.evaluate(X_test, Y_test)
print('RMSE: %.2f' % (result))